In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pandas_datareader.data as web
import talib as tlb
import seaborn as sns
import talib
from sklearn.metrics import confusion_matrix

In [2]:
start = dt.datetime(2014,1,1)
finish = dt.datetime.today()
source = 'yahoo'

In [3]:
df = web.DataReader('UNVR.JK', source, start, finish)

In [4]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-01-02,5360.0,5280.0,5280.0,5360.0,6175000.0,2777.326660
2014-01-03,5360.0,5240.0,5290.0,5300.0,8817500.0,2746.237305
2014-01-06,5360.0,5280.0,5360.0,5350.0,6357500.0,2772.144775
2014-01-07,5360.0,5240.0,5350.0,5240.0,7055500.0,2715.147949
2014-01-08,5315.0,5185.0,5280.0,5240.0,10381500.0,2715.147949


In [5]:
data = pd.DataFrame(index=df.index)
data["Today"] = df["Adj Close"]
#tslag["Volume"] = df["Volume"]

In [6]:
data['pct change']  = data['Today'].pct_change()

In [7]:
data['direction'] = np.sign(data['pct change'])

In [8]:
for n in [5, 14, 30, 50]:

    # Create the moving average indicator and divide by Adj_Close
    data['ma' + str(n)] = tlb.SMA(data["Today"].values,
                              timeperiod=n) / data["Today"]
    # Create the RSI indicator
    data['rsi' + str(n)] = tlb.RSI(data["Today"].values/data["Today"], timeperiod=n)

In [9]:
data.head(10)

,Today,pct change,direction,ma5,rsi5,ma14,rsi14,ma30,rsi30,ma50,rsi50
Date,,,,,,,,,,,
2014-01-02,2777.326660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,2746.237305,-0.011194,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,2772.144775,0.009434,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,2715.147949,-0.020561,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,2715.147949,0.000000,0.0,1.011069,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-09,2702.194092,-0.004771,-1.0,1.010355,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-10,2684.058594,-0.006711,-1.0,1.012548,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-13,2800.644043,0.043436,1.0,0.972433,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15,2904.275635,0.037003,1.0,0.950758,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data.dropna(inplace = True)

In [11]:
data.head()

,Today,pct change,direction,ma5,rsi5,ma14,rsi14,ma30,rsi30,ma50,rsi50
Date,,,,,,,,,,,
2014-03-17,3080.449463,-0.037247,-1.0,0.998318,0.0,0.966238,0.0,0.956574,0.0,0.943835,0.0
2014-03-18,3002.725830,-0.025231,-1.0,1.027610,0.0,0.993590,0.0,0.982629,0.0,0.969974,0.0
2014-03-19,2994.952881,-0.002589,-1.0,1.029585,0.0,0.998332,0.0,0.986073,0.0,0.973979,0.0
2014-03-20,2914.638672,-0.026817,-1.0,1.042489,0.0,1.024698,0.0,1.013156,0.0,1.002187,0.0
2014-03-21,2914.638672,0.000000,0.0,1.022933,0.0,1.024317,0.0,1.013037,0.0,1.003556,0.0


In [12]:
data.drop(['Today', 'pct change'], axis = 1, inplace = True)

In [13]:
data.isna().sum()

direction    0
ma5          0
rsi5         0
ma14         0
rsi14        0
ma30         0
rsi30        0
ma50         0
rsi50        0
dtype: int64

In [14]:
y = data['direction']

In [15]:
x = data.drop('direction', axis = 1)

In [16]:
ratio = 0.8
param = int(len(data)*ratio)
x_train = x[:param]
x_test = x[param:]
y_train = y[:param]
y_test = y[param:]

### KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [18]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Accuracy of kNN model: %0.3f" % model.score(x_test, y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, y_test))    

Accuracy of kNN model: 0.643
Confusion matrix: 
[[103   7  40]
 [  1   2   7]
 [ 33  17  84]]


In [19]:
y_train.isna().sum()

0

In [20]:
x_train.isna().sum()

ma5      0
rsi5     0
ma14     0
rsi14    0
ma30     0
rsi30    0
ma50     0
rsi50    0
dtype: int64

In [21]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Accuracy of kNN model: %0.3f" % model.score(x_test, y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, y_test))

Accuracy of kNN model: 0.643
Confusion matrix: 
[[103   7  40]
 [  1   2   7]
 [ 33  17  84]]


### Logistic Regression

In [22]:
model = LogisticRegression()

In [23]:
model.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
pred = model.predict(x_test)

In [25]:
print("Accuracy of logistic regression model: %0.3f" % model.score(x_test, y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, y_test))

Accuracy of logistic regression model: 0.571
Confusion matrix: 
[[84  7 47]
 [ 0  0  0]
 [53 19 84]]


### SVM

In [26]:
model = LinearSVC()
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Accuracy of SVM model: %0.3f" % model.score(x_test, y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, y_test))

Accuracy of SVM model: 0.636
Confusion matrix: 
[[97  7 41]
 [ 0  0  0]
 [40 19 90]]


### SVM 2

In [28]:
from sklearn.svm import SVC
model = SVC(C=1000000.0, cache_size=200,class_weight=None,coef0=0.0,
            degree=3,gamma=0.0001,kernel='rbf',
            max_iter=-1,probability=False,
            random_state=None,shrinking=True,tol=0.001,verbose=False)
model.fit(x_train, y_train)
pred = model.predict(x_test)
print("Accuracy of SVM model: %0.3f" % model.score(x_test, y_test))
print("Confusion matrix: \n%s" % confusion_matrix(pred, y_test))

Accuracy of SVM model: 0.721
Confusion matrix: 
[[117   8  36]
 [  0   0   0]
 [ 20  18  95]]
